Dependencies:
ms3 == 1.2.3
dimcat==0.0.post1.dev122+gd1e90a1
setuptools=65.6.3


In [1]:
import numpy as np
import pandas as pd
import ms3 as ms
from preprocessing import transform_chords_abs
import warnings

import dimcat as dc

warnings.simplefilter(action='ignore', category=FutureWarning)

pd.options.mode.chained_assignment = None

In [6]:
# Load Romantic Piano Corpus
#romantic_corpus = dc.Corpus()
#romantic_corpus.load("../../romantic_piano_corpus/grieg_lyric_pieces", parse_tsv=True, parse_scores=False) # make sure to parse directly from MuseScore files
#romantic_corpus.data

In [7]:
#test_corpus = dc.Corpus()

In [8]:
#test_corpus.load("../../romantic_piano_corpus/tchaikovsky_seasons", parse_tsv=True, parse_scores=False) # make sure to parse directly from MuseScore files

In [9]:
##romantic_corpus.load("../../romantic_piano_corpus/", parse_tsv=True, parse_scores=False) # make sure to parse directly from MuseScore files

In [10]:
#romantic_corpus.get_facet("notes")

In [4]:
# Load Romantic Piano Corpus
abc_corpus = dc.Dataset()
abc_corpus.load("../../ABC/", parse_tsv=True, parse_scores=False) # make sure to parse directly from MuseScore files
#romantic_corpus.data

AttributeError: module 'dimcat' has no attribute 'Dataset'

In [5]:
abc_corpus.data

NameError: name 'abc_corpus' is not defined

In [13]:
abc_corpus.get_facet("expanded")

quarterbeats  duration_qb   mc   mn   
corpus fname        interval                                               
ABC    n01op18-1_01 [0.0, 3.0)                  0          3.0    1    1  \
                    [3.0, 6.0)                  3          3.0    2    2   
                    [6.0, 9.0)                  6          3.0    3    3   
                    [9.0, 15.0)                 9          6.0    4    4   
                    [15.0, 18.0)               15          3.0    6    6   
...                                           ...          ...  ...  ...   
       n16op135_04  [1140.0, 1141.0)         1140          1.0  281  280   
                    [1141.0, 1142.0)         1141          1.0  281  280   
                    [1142.0, 1144.0)         1142          2.0  282  281   
                    [1144.0, 1146.0)         1144          2.0  282  281   
                    [1146.0, 1150.0)         1146          4.0  283  282   

                                     mc_onset mn_onset timesig  staff  voice   
corpus fname        interval                                                   
ABC    n01op18-1_01 [0.0, 3.0)              0        0     3/4      4      1  \
                    [3.0, 6.0)              0        0     3/4      4      1   
                    [6.0, 9.0)              0        0     3/4      4      1   
                    [9.0, 15.0)             0        0     3/4      4      1   
                    [15.0, 18.0)            0        0     3/4      4      1   
...                                       ...      ...     ...    ...    ...   
       n16op135_04  [1140.0, 1141.0)      1/2      1/2     4/4      4      1   
                    [1141.0, 1142.0)      3/4      3/4     4/4      4      1   
                    [1142.0, 1144.0)        0        0     4/4      4      1   
                    [1144.0, 1146.0)      1/2      1/2     4/4      4      1   
                    [1146.0, 1150.0)        0        0     4/4      4      1   

                                      volta  ... phraseend chord_type   
corpus fname        interval                 ...                        
ABC    n01op18-1_01 [0.0, 3.0)         <NA>  ...       NaN          M  \
                    [3.0, 6.0)         <NA>  ...       NaN          M   
                    [6.0, 9.0)         <NA>  ...       NaN          M   
                    [9.0, 15.0)        <NA>  ...       NaN          M   
                    [15.0, 18.0)       <NA>  ...       NaN        Mm7   
...                                     ...  ...       ...        ...   
       n16op135_04  [1140.0, 1141.0)   <NA>  ...       NaN          M   
                    [1141.0, 1142.0)   <NA>  ...       NaN          M   
                    [1142.0, 1144.0)   <NA>  ...       NaN          M   
                    [1144.0, 1146.0)   <NA>  ...       NaN        Mm7   
                    [1146.0, 1150.0)   <NA>  ...        \\          M   

                                     globalkey_is_minor localkey_is_minor   
corpus fname        interval                                                
ABC    n01op18-1_01 [0.0, 3.0)                    False             False  \
                    [3.0, 6.0)                    False             False   
                    [6.0, 9.0)                    False             False   
                    [9.0, 15.0)                   False             False   
                    [15.0, 18.0)                  False             False   
...                                                 ...               ...   
       n16op135_04  [1140.0, 1141.0)               True             False   
                    [1141.0, 1142.0)               True             False   
                    [1142.0, 1144.0)               True             False   
                    [1144.0, 1146.0)               True             False   
                    [1146.0, 1150.0)               True             False   

                                        chord_tones add

# Helper Functions


In [14]:
# Given a corpus, process all pieces and return labels and slices
def processCorpus(corpus):
    labels = corpus.get_facet("expanded")
    # Process slices 
    salami_crp = dc.NoteSlicer().process_data(corpus)
    salami_notes = salami_crp.get_facet("notes")

    return (labels, salami_notes)

def to_pitch(midi, tpc):
    pitch_class = ms.fifths2name(tpc)
    octave = str(midi // 12)
    return pitch_class + octave 

# Returns the offset using semitones
def get_chord_offset(numeral: str, globalkey_is_minor):
    alteration = (numeral.count("#") - numeral.count("b")) * 7
    #print(alteration)

    numeral = numeral.strip("#b")
    numeral = numeral.upper()
    #numeral_to_interval_major = {"I": 0, "II": 2, "III": 4, "IV": 5, "V":7, "VI":9, "VII":11}
    #numeral_to_interval_minor = {"I": 0, "II": 2, "III": 3, "IV": 5, "V":7, "VI":8, "VII":10}
    numeral_to_interval_major = {"I": 0, "II": 2, "III": 4, "IV": -1, "V":1, "VI":3, "VII":8}
    numeral_to_interval_minor = {"I": 0, "II": 2, "III": -3, "IV": -1, "V":1, "VI":8, "VII":-2}

    if globalkey_is_minor:
        return (numeral_to_interval_minor[numeral] + alteration) % 12
    else:
        return (numeral_to_interval_major[numeral] + alteration) % 12

def interval_union(i1,i2):
    return pd.Interval(i1.left,i2.right,'left')

def transform_chords_abs(df):
    df['rootoffset'] = df.apply(lambda x: int(get_chord_offset(x.numeral,x.globalkey_is_minor)), axis = 1)

In [102]:
# Returns two dataframes, one for the chords, one for the slices
def preprocessPiece(piece : str, labels, salami_notes):
    # zoom in on the chords in one piece
    chords = labels.loc[('ABC', piece)]

    # Translate labels to absolute pitches

    desired_chord_columns = ['chord','pedal','numeral','form','figbass','changes','relativeroot','localkey','globalkey']
    chordz = chords.copy().reset_index()
    chordz = chordz.reset_index()

    chords_abs_columns = ['chord', 'globalkey','globalkey_is_minor']

    clean_chords = chordz[chordz['chord'] != '@none']
    ms.labels2global_tonic(clean_chords, inplace=True)
    clean_chords.to_csv("chordsbefore.csv")
    transform_chords_abs(clean_chords)

    # Recombine the segments with @None labels
    full_chords_abs = pd.concat([clean_chords, chordz[chordz['chord'] == '@none']]).sort_index()
    full_chords_abs.rootoffset.fillna(0, inplace=True)

    # Now we merge repeated chords
    relavant_columns = ["interval", "chord_type", "rootoffset", "globalkey"]

    dfs = pd.DataFrame()
    ind = 0
    prev = None 
    for row in full_chords_abs[relavant_columns].iterrows():
        v = row[1]
        ii = len(dfs.index) - 1
        if prev and (v.chord_type == prev[1].chord_type and v.rootoffset == prev[1].rootoffset):
            dfs.at[ii, 'interval'] = pd.Interval(dfs.iloc[ii].interval.left, v.interval.right, "left")
        else:
            new_row = pd.DataFrame({'interval':row[1].interval, 'chord_type':v.chord_type,'rootoffset':v.rootoffset,'globalkey':v.globalkey},index=[ind])
            dfs = pd.concat([dfs, new_row])
            ind += 1
    prev = row

    dfs.rootoffset = dfs.rootoffset.astype(int)
    full_chords_abs = dfs

    relavant_columns = [ "interval", "chord_type", "rootoffset", "globalkey"]

    full_chords_abs = full_chords_abs.reset_index()[relavant_columns]
    full_chords_abs.index.name ='segment_id'
    full_chords_abs[["chord_type", "rootoffset", "globalkey"]].to_csv('chords.csv')


    salamis = salami_notes.loc[("ABC", piece)]

    mini_salamis = salamis[['midi','tpc','tied']]
    mini_salamis['tied'] = mini_salamis['tied'].fillna(0).astype('bool')

    # Assigning each slice a segment id according to the chord.
    dfs = []
    for segment, interval in enumerate(full_chords_abs["interval"]):
        segMask = mini_salamis.index.get_level_values(0).overlaps(interval)
        slicesInInterval = mini_salamis[segMask]
        slicesInInterval.insert(0,'segment_id',segment)
        dfs.append(slicesInInterval)

    segmented_salamis = pd.concat(dfs)

    segmented_salamis['slice_id'] = pd.factorize(segmented_salamis.reset_index()['onset_slice'])[0]

    segmented_salamis['pitch'] = segmented_salamis.apply(lambda x: to_pitch(x.midi, x.tpc), axis=1)

    final_salamis_columns = ['segment_id','slice_id','pitch','tied']
    final_salamis = segmented_salamis.reset_index()[final_salamis_columns]

    final_salamis["new_segment"] = final_salamis["segment_id"].diff().astype(bool)
    final_salamis['new_slice'] = final_salamis["slice_id"].diff().astype(bool)


    # Capitalise Global key to fix bug with Haskell Musicology with lowercase b
    full_chords_abs.globalkey = full_chords_abs.globalkey.apply(lambda key: key.capitalize())

    # Correct the new segment and new slice fields for the first row.
    final_salamis.at[0, "new_segment"] = False
    final_salamis.at[0, "new_slice"] = False

    final_salamis.to_csv('salamis.csv',columns=["new_segment", "new_slice", "pitch","tied"], index=False)

    return (full_chords_abs[["chord_type", "rootoffset", "globalkey"]], final_salamis[["new_segment", "new_slice", "pitch", "tied"]])




In [16]:
(labels, salamis) = processCorpus(abc_corpus)

In [17]:
pieces = labels.loc["ABC"].index.unique(0).tolist()

In [19]:
piece = pieces[0]

In [25]:
# Returns two dataframes, one for the chords, one for the slices
# zoom in on the chords in one piece
chords = labels.loc[('ABC', piece)]

# Translate labels to absolute pitches

desired_chord_columns = ['chord','pedal','numeral','form','figbass','changes','relativeroot','localkey','globalkey']
chordz = chords.copy().reset_index()
chordz = chordz.reset_index()

chords_abs_columns = ['chord', 'globalkey','globalkey_is_minor']

clean_chords = chordz[chordz['chord'] != '@none']
ms.labels2global_tonic(clean_chords, inplace=True)
clean_chords.to_csv("chordsbefore.csv")
transform_chords_abs(clean_chords)

# Recombine the segments with @None labels
full_chords_abs = pd.concat([clean_chords, chordz[chordz['chord'] == '@none']]).sort_index()
full_chords_abs.rootoffset.fillna(0, inplace=True)

# Now we merge repeated chords
relavant_columns = ["interval", "chord_type", "rootoffset", "globalkey"]






In [43]:
full_chords_abs[relavant_columns]

,interval,chord_type,rootoffset,globalkey
0,"[0.0, 3.0)",M,0.0,F
1,"[3.0, 6.0)",M,1.0,F
2,"[6.0, 9.0)",M,0.0,F
3,"[9.0, 15.0)",M,11.0,F
4,"[15.0, 18.0)",Mm7,1.0,F
...,...,...,...,...
400,"[921.0, 924.0)",M,0.0,F
401,"[924.0, 927.0)",Fr,2.0,F
402,"[927.0, 930.0)",M,0.0,F
403,"[930.0, 937.0)",M,0.0,F


In [57]:
relavant_columns

['interval', 'chord_type', 'rootoffset', 'globalkey']

In [101]:
dfs.tail(1).index[0]

324

In [95]:
dfs = pd.DataFrame()
ind = 0
prev = None 
for row in full_chords_abs[relavant_columns].iterrows():
    #print("DF:")
    #print(dfs)
    #print("")
    v = row[1]
    ii = len(dfs.index) - 1
    #print(row)
    if prev and (v.chord_type == prev[1].chord_type and v.rootoffset == prev[1].rootoffset):
        # combine
        #print("Combine")
        #print(v)
        #print("With")
        #print(dfs.iloc[ii])
        #print("Result")

        dfs.at[ii, 'interval'] = pd.Interval(dfs.iloc[ii].interval.left, v.interval.right, "left")
        #print(dfs.iloc[-1]) 
    else:
        # New row
        # new = pd.DataFrame(row[1], columns=relavant_columns)
        new_row = pd.DataFrame({'interval':row[1].interval, 'chord_type':v.chord_type,'rootoffset':v.rootoffset,'globalkey':v.globalkey},index=[ind])
        #print(new_row)
        #print()
        dfs = pd.concat([dfs, new_row])
        ind += 1
    prev = row

DF:
Empty DataFrame
Columns: []
Index: []

     interval chord_type  rootoffset globalkey
0  [0.0, 3.0)          M         0.0         F

DF:
     interval chord_type  rootoffset globalkey
0  [0.0, 3.0)          M         0.0         F

     interval chord_type  rootoffset globalkey
1  [3.0, 6.0)          M         1.0         F

DF:
     interval chord_type  rootoffset globalkey
0  [0.0, 3.0)          M         0.0         F
1  [3.0, 6.0)          M         1.0         F

     interval chord_type  rootoffset globalkey
2  [6.0, 9.0)          M         0.0         F

DF:
     interval chord_type  rootoffset globalkey
0  [0.0, 3.0)          M         0.0         F
1  [3.0, 6.0)          M         1.0         F
2  [6.0, 9.0)          M         0.0         F

      interval chord_type  rootoffset globalkey
3  [9.0, 15.0)          M        11.0         F

DF:
      interval chord_type  rootoffset globalkey
0   [0.0, 3.0)          M         0.0         F
1   [3.0, 6.0)          M         1.0

In [96]:
dfs

,interval,chord_type,rootoffset,globalkey
0,"[0.0, 3.0)",M,0.0,F
1,"[3.0, 6.0)",M,1.0,F
2,"[6.0, 9.0)",M,0.0,F
3,"[9.0, 15.0)",M,11.0,F
4,"[15.0, 18.0)",Mm7,1.0,F
...,...,...,...,...
320,"[915.0, 918.0)",M,0.0,F
321,"[918.0, 921.0)",m,11.0,F
322,"[921.0, 924.0)",M,0.0,F
323,"[924.0, 927.0)",Fr,2.0,F


In [ ]:

dfs.rootoffset = dfs.rootoffset.astype(int)
full_chords_abs = dfs

relavant_columns = [ "interval", "chord_type", "rootoffset", "globalkey"]

full_chords_abs = full_chords_abs.reset_index()[relavant_columns]
full_chords_abs.index.name ='segment_id'
full_chords_abs[["chord_type", "rootoffset", "globalkey"]].to_csv('chords.csv')


salamis = salami_notes.loc[("ABC", piece)]

mini_salamis = salamis[['midi','tpc','tied']]
mini_salamis['tied'] = mini_salamis['tied'].fillna(0).astype('bool')

# Assigning each slice a segment id according to the chord.
dfs = []
for segment, interval in enumerate(full_chords_abs["interval"]):
    segMask = mini_salamis.index.get_level_values(0).overlaps(interval)
    slicesInInterval = mini_salamis[segMask]
    slicesInInterval.insert(0,'segment_id',segment)
    dfs.append(slicesInInterval)

segmented_salamis = pd.concat(dfs)

segmented_salamis['slice_id'] = pd.factorize(segmented_salamis.reset_index()['onset_slice'])[0]

segmented_salamis['pitch'] = segmented_salamis.apply(lambda x: to_pitch(x.midi, x.tpc), axis=1)

final_salamis_columns = ['segment_id','slice_id','pitch','tied']
final_salamis = segmented_salamis.reset_index()[final_salamis_columns]

final_salamis["new_segment"] = final_salamis["segment_id"].diff().astype(bool)
final_salamis['new_slice'] = final_salamis["slice_id"].diff().astype(bool)


# Capitalise Global key to fix bug with Haskell Musicology with lowercase b
full_chords_abs.globalkey = full_chords_abs.globalkey.apply(lambda key: key.capitalize())

# Correct the new segment and new slice fields for the first row.
final_salamis.at[0, "new_segment"] = False
final_salamis.at[0, "new_slice"] = False

final_salamis.to_csv('salamis.csv',columns=["new_segment", "new_slice", "pitch","tied"], index=False)

x = full_chords_abs[["chord_type", "rootoffset", "globalkey"]]
y = final_salamis[["new_segment", "new_slice", "pitch", "tied"]]

In [105]:
for piece in pieces[0:10]:
    (chords, slices) = preprocessPiece(piece, labels, salamis)
    chords.to_csv("inputs/chords/{}.csv".format(piece))
    slices.to_csv("inputs/slices/{}.csv".format(piece),index=False)

# EXPERIMENTS

In [110]:
import subprocess
import json

In [104]:
def runFullParse (piece:str):
    jsonPath = "outputs/"+piece+".json"
    cmd = ["stack","run","fullParse","--","-n","10",piece, "All"] 
    print("Running command: " + (" ".join(cmd)))
    print("Expecting results in " + jsonPath)
    #res = subprocess.run(cmd, cwd="..")
    res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, cwd="..")

    if res.returncode != 0:
        print("Error in subprocess")
        print(res.stderr)
        print(res.stdout)
        return;
    else:
        f = open(jsonPath)
        results = json.load(f)
        f.close()
        return results
     

In [106]:
def runExperiment(piece:str): 
    return runFullParse(piece)
def printResults(res):
    for (k,v) in res.items():
        print("{}: {}".format(k, v))


In [107]:
def runFullPieceExperiment(pieces):
    # Runs Heuristic search, Random Search, and Random sample search on the given piece.
    # Returns the scores for each 
    res = []
    for piece in pieces:
      res.append(runFullParse(piece))
    return res 



In [ ]:
res = runFullPieceExperiment(pieces[0:3])